In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

 # linear algebra
 # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
from tensorflow.python.keras import Sequential
from tensorflow.keras import layers, optimizers, applications, callbacks
from tensorflow.keras.applications import DenseNet121  
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler
from IPython.display import display
from tensorflow.keras import backend as K
import tensorflow.keras.preprocessing.image as tf_image
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

In [ ]:
train = pd.read_csv('/kaggle/input/facial-keypoints-detection/training.zip')
test = pd.read_csv('/kaggle/input/facial-keypoints-detection/test.zip')
idlookup_data = pd.read_csv('/kaggle/input/facial-keypoints-detection/IdLookupTable.csv')

In [ ]:
test.info()

In [ ]:
idlookup_data.info()
idlookup_data.head()

In [ ]:
test.head()

In [ ]:
test.info()

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
train.isnull().sum()

In [ ]:
train_image = train.Image
train = train.drop('Image', axis=1)


In [ ]:
train_img = train_image.apply( lambda x: np.fromstring(x, dtype=float, sep=' ').reshape(96, 96) )

type(train_img[0])
train_img[0].shape

In [ ]:

train_data = np.array(train).reshape(7049, 15,2)


In [ ]:
plt.imshow(train_img[0], cmap='gray')
plt.plot(train_data[0][:, 0], train_data[0][:, 1], 'rx')

In [ ]:
plt.imshow(train_img[7048], cmap='gray')
plt.plot(train_data[7048][:, 0], train_data[0][:, 1], 'rx')

In [ ]:
train.isnull().sum()


# Augmentation

#### Horizontal and Vertical Flip

In [ ]:
def horizontal_flip(df, col):
    image, keypoints = df[-1:], df[:-1]
    image1 = np.array(image)[0]
    n = int(keypoints.shape[0]/2)
    keypoints1 = np.array(keypoints).reshape(-1, n, 2)[0]
    
    image2 = np.flip(image1, axis=1)
    keypoints1[:, 0] = 96-keypoints1[:, 0]
    
    keypoints2 = keypoints1.reshape(-1, 2*n)[0]
    s = pd.Series(keypoints2, index=col[:-1])
    s['Image'] = image2
    
    return s

def vertical_flip(df, col):
    image, keypoints = df[-1:], df[:-1]
    image1 = np.array(image)[0]
    n = int(keypoints.shape[0]/2)
    keypoints1 = np.array(keypoints).reshape(-1, n, 2)[0]
    
    image2 = np.flip(image1, axis=0)
    keypoints1[:, 1] = 96-keypoints1[:, 1]
    
    keypoints2 = keypoints1.reshape(-1, 2*n)[0]
    s = pd.Series(keypoints2, index=col[:-1])
    s['Image'] = image2
    
    return s


#### Display Function

In [ ]:
def display(image, keypoints=[]):
    plt.figure()
    plt.imshow(image, cmap='gray')
    if len(keypoints)>0:
        plt.plot(keypoints[:, 0], keypoints[:, 1], 'rx')
    
def compare(image1, keypoints1, image2, keypoints2):
    plt.figure()
    plt.subplot(1,2,1)
    plt.imshow(image1, cmap='gray')
    plt.plot(keypoints1[:, 0], keypoints1[:, 1], 'rx')
    
    plt.subplot(1,2,2)
    plt.imshow(image2, cmap='gray')
    plt.plot(keypoints2[:, 0], keypoints2[:, 1], 'rx')
    
    
def df_plot(df):
    fig = plt.figure(figsize=(24,24))
    for i in range(6):
        ax = fig.add_subplot(1, 6, i+1)
        image = plt.imshow(df['Image'][i], cmap = 'gray')
        n = int(df.iloc[i][:-1].shape[0]) + 1
        for j in range(1, n, 2):
            plt.plot(df.loc[i][j-1], df.loc[i][j], 'rx')

#### Rotation

In [ ]:
def rotation(df, col):  
    image, keypoints = df[-1:], df[:-1]
    image1 = np.array(image)[0]
    n = int(keypoints.shape[0]/2)
    keypoints1 = np.array(keypoints).reshape(-1, n, 2)[0]
    
    angle = np.random.uniform(-60, 60)
    h,w = 96, 96
    cX, cY = 48, 48
    
    M = cv2.getRotationMatrix2D((cX, cY), angle, 1)
    
    #  
    #  (rot_mat) M =  [ m00    m01  m02]   = [ cosx    sinx  (1−cosx)cX − sinx*cY ]
    #                 [m10    m11  m12]      [-sinx   cosx   sinx*cX + (1−cosx)cY ]     
    #
    
    cos = np.abs(M[0][0])
    sin = np.abs(M[0][1])

    #     nW = int(h*sin + w*cos)
    #     nH = int(w*sin + h*cos)
    
    nW = 96
    nH = 96

    #     M[0, 2] += (nW / 2) - cX
    #     M[1, 2] += (nH / 2) - cY
    
    r_img = cv2.warpAffine(image1, M, (nW, nH))
    
    keypoints1[:, 0] = keypoints1[:, 0] - cX
    keypoints1[:, 1] = keypoints1[:, 1] - cY

    r_angl = np.radians(angle)
    cos = np.cos(r_angl)
    sin = np.sin(r_angl)

    M2 = np.array([[cos, sin],
                  [-sin, cos]])
    
    r_keypoints = np.dot(M2, keypoints1.T)
    r_keypoints = r_keypoints.T
    r_keypoints[:, 0] = r_keypoints[:, 0] + (nW)/2.0
    r_keypoints[:, 1] = r_keypoints[:, 1] + (nH)/2.0
    
    keypoints2 = r_keypoints.reshape(-1, 2*n)[0]
    s = pd.Series(keypoints2, index=col[:-1])
    s['Image'] = r_img
    
    return s
    


#### Brightness

In [ ]:
def brightness(df, col):
    image, keypoints = df[-1:], df[:-1]
    image1 = np.array(image)[0]
    n = int(keypoints.shape[0]/2)
    keypoints1 = np.array(keypoints).reshape(-1, n, 2)[0]
    
    image2 = np.clip(np.random.uniform(0.1, 3)*image1, 0.0, 255.0)
    
    keypoints2 = keypoints1.reshape(-1, 2*n)[0]
    s = pd.Series(keypoints2, index=col[:-1])
    s['Image'] = image2
    
    return s


##### Shift

In [ ]:
def shift(df, col):
    image, keypoints = df[-1:], df[:-1]
    image1 = np.array(image)[0]
    n = int(keypoints.shape[0]/2)
    keypoints1 = np.array(keypoints).reshape(-1, n, 2)[0]
    
    tx = np.random.uniform(-10, 10)
    ty = np.random.uniform(-10, 10)
    
    M = np.array([[1, 0, tx],
                  [0, 1, ty]])
    
    image2 = cv2.warpAffine(image1, M, (96, 96))
    
    keypoints1[:, 0] = keypoints1[:, 0] + tx 
    keypoints1[:, 1] = keypoints1[:, 1] + ty 
    
    keypoints2 = keypoints1.reshape(-1, 2*n)[0]
    s = pd.Series(keypoints2, index=col[:-1])
    s['Image'] = image2
    return s



#### Noise and Blur

In [ ]:
def noise(df, col):
    image, keypoints = df[-1:], df[:-1]
    image1 = np.array(image)[0]
    n = int(keypoints.shape[0]/2)
    keypoints1 = np.array(keypoints).reshape(-1, n, 2)[0]
    
    image2 = cv2.add(image1, np.random.uniform(0.1,0.9)*np.random.randint(50, size=(96,96)))
    
    keypoints2 = keypoints1.reshape(-1, 2*n)[0]
    s = pd.Series(keypoints2, index=col[:-1])
    s['Image'] = image2
    return s




In [ ]:
def blur(df, col):
    image, keypoints = df[-1:], df[:-1]
    image1 = np.array(image)[0]
    n = int(keypoints.shape[0]/2)
    keypoints1 = np.array(keypoints).reshape(-1, n, 2)[0]
    
    image2 = cv2.GaussianBlur(image1, (5,5), 0)
    
    keypoints2 = keypoints1.reshape(-1, 2*n)[0]
    s = pd.Series(keypoints2, index=col[:-1])
    s['Image'] = image2
    return s



In [ ]:
df = pd.read_csv('/kaggle/input/facial-keypoints-detection/training.zip')

In [ ]:
df['Image'] = df['Image'].apply(lambda x: np.fromstring(x, dtype=float, sep=' ').reshape(96,96))

In [ ]:
c_df = df.dropna()
c_df.head()

In [ ]:
c_df.info()

In [ ]:
df.isnull().sum()

In [ ]:
tf_ = c_df.copy()

In [ ]:
gf = c_df[:10].copy()

In [ ]:
df_plot(gf)

In [ ]:
col = tf_.columns
df_plot(tf_)

In [ ]:
tf1 = tf_.apply(lambda row: horizontal_flip(row, col), axis=1)
df_plot(tf1)

In [ ]:
tf2 = tf_.apply(lambda row: vertical_flip(row, col), axis=1)
df_plot(tf2)

In [ ]:
tf3 = tf_.apply(lambda row: rotation(row, col), axis=1)
df_plot(tf3)

In [ ]:
tf4 = tf_.apply(lambda row: brightness(row, col), axis=1)
df_plot(tf4)

In [ ]:
tf5 = tf_.apply(lambda row: blur(row, col), axis=1)
df_plot(tf5)

In [ ]:
tf6 = tf_.apply(lambda row: noise(row, col), axis=1)
df_plot(tf6)

In [ ]:
tf7 = tf_.apply(lambda row: shift(row, col), axis=1)
df_plot(tf7)

In [ ]:
col = c_df.columns

In [ ]:
c_df = c_df.sample(frac=1).reset_index(drop=True)
c_df.head(1)

In [ ]:
df_plot(c_df)

In [ ]:
c_df.shape

In [ ]:
n = int(len(c_df)/2)
c_df11 = c_df.iloc[:n].reset_index(drop=True)
c_df11 = c_df11.copy()

c_df12 = c_df.iloc[n:].reset_index(drop=True)
c_df12 = c_df12.copy()

In [ ]:
c_df11 = c_df11.apply(lambda row: vertical_flip(row, col), axis=1)
# c_df = c_df.append(c_df11, ignore_index=True)

c_df12 = c_df12.apply(lambda row: horizontal_flip(row, col), axis=1)
# c_df = c_df.append(c_df12, ignore_index=True)

c_df = pd.concat([c_df,c_df11, c_df12], ignore_index=True)


In [ ]:
c_df = c_df.sample(frac=1).reset_index(drop=True)
c_df.shape

In [ ]:
c_df21 = c_df.copy()
c_df21 = c_df21.apply(lambda row: rotation(row, col), axis=1)
c_df = c_df.append(c_df21, ignore_index=True)

In [ ]:
c_df.shape

In [ ]:
c_df = c_df.sample(frac=1).reset_index(drop=True)

n = int(len(c_df)/4)
c_df31 = c_df.iloc[:n].reset_index(drop=True)
c_df31 = c_df31.copy()

c_df31 = c_df31.apply(lambda row: shift(row, col), axis=1)
c_df = c_df.append(c_df31, ignore_index=True)

In [ ]:
c_df.shape

In [ ]:
c_df = c_df.sample(frac=1).reset_index(drop=True)
n = int(len(c_df)/2)
c_df41 = c_df.iloc[:n].reset_index(drop=True)
c_df41 = c_df41.copy()

c_df41 = c_df41.apply(lambda row: brightness(row, col), axis=1)
c_df = c_df.append(c_df41, ignore_index=True)

In [ ]:
c_df.shape

In [ ]:
c_df = c_df.sample(frac=1).reset_index(drop=True)
n = int(len(c_df)/3)
c_df51 = c_df.iloc[:n].reset_index(drop=True)
c_df51 = c_df51.copy()

c_df52 = c_df.iloc[n:2*n].reset_index(drop=True)
c_df52 = c_df52.copy()

In [ ]:
c_df51 = c_df51.apply(lambda row: noise(row, col), axis=1)
c_df52 = c_df52.apply(lambda row: blur(row, col), axis=1)
c_df = pd.concat([c_df,c_df51, c_df52], ignore_index=True)

In [ ]:
c_df = c_df.sample(frac=1).reset_index(drop=True)

In [ ]:
c_df.shape

In [ ]:
# c_df = c_df.sample(frac=1).reset_index(drop=True)
# df_plot(c_df)

In [ ]:
d_df = df[['left_eye_center_x', 'left_eye_center_y', 'right_eye_center_x', 'right_eye_center_y', 'nose_tip_x', 'nose_tip_y', 'mouth_center_bottom_lip_x', 'mouth_center_bottom_lip_y', 'Image']]
d_df.info()


In [ ]:
d_df = d_df.dropna()
d_df.isnull().sum()

In [ ]:
d_df.shape

In [ ]:
d_tf = d_df.copy()
d_tf.head(1)

In [ ]:
a = d_tf.iloc[0][:-1].shape[0]
a

In [ ]:
display(d_tf.iloc[0, -1])

In [ ]:
dtf = d_tf.sample(frac=1).reset_index(drop=True)
dcol = dtf.columns

In [ ]:
dcol

In [ ]:

dtf1 = dtf.iloc[:3000].copy()
dtf2 = dtf.iloc[3000:6000].copy()



In [ ]:
dtf1 = dtf1.apply(lambda row: horizontal_flip(row, dcol), axis=1)
dtf2 = dtf2.apply(lambda row: vertical_flip(row, dcol), axis=1)

dtf = pd.concat([dtf,dtf1, dtf2], ignore_index=True)

In [ ]:
dtf = dtf.sample(frac=1).reset_index(drop=True)
dtf.shape

In [ ]:
n1 = int(len(dtf)/2)
dtf3 = dtf.iloc[:n].copy()
dtf3 = dtf3.apply(lambda row: rotation(row, dcol), axis=1)
dtf = pd.concat([dtf, dtf3], ignore_index=True)

In [ ]:
dtf = dtf.sample(frac=1).reset_index(drop=True)
dtf.shape

In [ ]:
n1 = int(len(dtf)/4)

dtf4 = dtf.iloc[:n1].copy()
dtf5 = dtf.iloc[n1:2*n1].copy()
dtf6 = dtf.iloc[2*n1:3*n1].copy()
dtf7 = dtf.iloc[3*n1:].copy()

In [ ]:
dtf4 = dtf4.apply(lambda row: brightness(row, dcol), axis=1)
dtf5 = dtf5.apply(lambda row: noise(row, dcol), axis=1)
dtf6 = dtf6.apply(lambda row: blur(row, dcol), axis=1)
dtf7 = dtf7.apply(lambda row: shift(row, dcol), axis=1)


dtf = pd.concat([dtf,dtf4, dtf5, dtf6, dtf7], ignore_index=True)

In [ ]:
dtf.shape

### Normalization and training data preparation

In [ ]:
img = c_df.iloc[:, -1]
img = img/255.0

# Create an empty array of shape (32k, 96, 96, 1) to train the model
X = np.empty((len(img), 96, 96, 1))

# Iterate through the normalized images list and add image values to the empty array 
# Note that we need to expand it's dimension from (96,96) to (96,96,1)
for i in range(len(img)):
    X[i,] = np.expand_dims(img[i], axis = 2)

# Convert the array type to float32
X = np.asarray(X).astype(np.float32)
X.shape

In [ ]:
y = c_df.iloc[:,:-1]
y = np.asarray(y).astype(np.float32)
y.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.15)

# Model

In [ ]:
def create_model(output=30):
    model = Sequential()

    pretrained_model = applications.MobileNet(input_shape=(96, 96, 3), include_top=False, weights='imagenet')
    pretrained_model.trainable = True

    model.add(layers.Convolution2D(3, (1, 1), padding='same', input_shape=(96,96,1)))
    model.add(layers.LeakyReLU(alpha = 0.1))
    model.add(pretrained_model)
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dropout(0.1))
    model.add(layers.Dense(output))
    return model

# model.summary()

In [ ]:
model_1 = create_model()
model_1.summary()

In [ ]:
model_1.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

In [ ]:
history = model_1.fit(X_train, y_train, batch_size = 256, epochs= 100, validation_split = 0.05)

In [ ]:
result = model_1.evaluate(X_test,y_test)
print("Accuracy : {}".format(result[1]))

In [ ]:
# Getting the model history keys 
history.history.keys()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train_loss','val_loss'], loc = 'upper right')
plt.show()

In [ ]:
df_predict = model_1.predict(X_test)

from sklearn.metrics import mean_squared_error
from math import sqrt

rms = sqrt(mean_squared_error(y_test, df_predict))
print("RMSE value : {}".format(rms))

In [ ]:
df_predict= pd.DataFrame(df_predict)
df_predict.head()

In [ ]:
fig = plt.figure(figsize=(20, 20))
for i in range(8):
    ax = fig.add_subplot(4, 2, i + 1)
    # Using squeeze to convert the image shape from (96,96,1) to (96,96)
    plt.imshow(X_test[i].squeeze(),cmap='gray')
    for j in range(1,31,2):
            plt.plot(df_predict.loc[i][j-1], df_predict.loc[i][j], 'rx')

In [ ]:
model_1.save("my_model.h5")

In [ ]:
test_image = test.iloc[:, -1].copy()

In [ ]:
test_image

In [ ]:
test_image = test_image.apply(lambda x: np.fromstring(x, dtype=float, sep=' ').reshape(96, 96))

In [ ]:
test_image

In [ ]:
imgt = test_image
imgt = imgt/255.0

# Create an empty array of shape (32k, 96, 96, 1) to train the model
Xt = np.empty((len(imgt), 96, 96, 1))

# Iterate through the normalized images list and add image values to the empty array 
# Note that we need to expand it's dimension from (96,96) to (96,96,1)
for i in range(len(imgt)):
    Xt[i,] = np.expand_dims(imgt[i], axis = 2)

# Convert the array type to float32
Xt = np.asarray(Xt).astype(np.float32)
Xt.shape

In [ ]:
test_preds = model_1.predict(Xt)

In [ ]:
pd.DataFrame(test_preds).to_csv('test_preds4_1.csv',index = False)

In [ ]:
fig = plt.figure(figsize=(20, 20))

for i in range(64):
    ax = fig.add_subplot(8, 8, i + 1)    
    image = plt.imshow(Xt[i].reshape(96,96), cmap = 'gray')
    for j in range(1,31,2):
        plt.plot(test_preds[i][j-1], test_preds[i][j], 'rx')
    

In [ ]:
feature_names = list(idlookup_data['FeatureName'])
image_ids = list(idlookup_data['ImageId']-1)
row_ids = list(idlookup_data['RowId'])

feature_list = []
for feature in feature_names:
    feature_list.append(feature_names.index(feature))
    
predictions = []
for x,y in zip(image_ids, feature_list):
    predictions.append(test_preds[x][y])
    
row_ids = pd.Series(row_ids, name = 'RowId')
locations = pd.Series(predictions, name = 'Location')
locations = locations.clip(0.0,96.0)
submission_result = pd.concat([row_ids,locations],axis = 1)
submission_result.to_csv('submission4_1.csv',index = False)